In [1]:
import fitz  # PyMuPDF
import re
import os

In [5]:
import fitz  # PyMuPDF
import re
import os

def split_pdf_based_on_pattern(input_pdf_path, output_directory, pattern):
    # Open the input PDF using fitz
    doc = fitz.open(input_pdf_path)
    total_pages = doc.page_count
    
    # Regex pattern to detect the booklet start (e.g., '8392932-----238932')
    booklet_start_pattern = re.compile(pattern)
    # Regex pattern to extract the customer ID (number before the dashes)
    customer_id_pattern = re.compile(r"(\d+)-----")

    # Initialize variables
    current_booklet_pages = []
    current_customer_id = None

    # Ensure output directory exists
    os.makedirs(output_directory, exist_ok=True)

    def save_booklet(pages, customer_id):
        if not pages:
            return
        # Create a new PDF
        new_pdf = fitz.open()
        for page_num in pages:
            new_pdf.insert_pdf(doc, from_page=page_num, to_page=page_num)
        
        # Sanitize the customer ID for filenames
        sanitized_id = re.sub(r'[<>:"/\\|?*]', '_', customer_id)
        output_pdf_path = os.path.join(output_directory, f"{sanitized_id}.pdf")
        new_pdf.save(output_pdf_path)
        new_pdf.close()  # Close the new PDF to free resources
        print(f"Saved booklet '{sanitized_id}' with {len(pages)} pages.")

    # Iterate through all the pages in the PDF
    for i in range(total_pages):
        page = doc.load_page(i)
        text = page.get_text()

        # Check if the page contains the booklet start pattern
        match = booklet_start_pattern.search(text)
        if match:
            # Extract customer ID from the matched text
            customer_id_match = customer_id_pattern.search(text)
            if customer_id_match:
                customer_id = customer_id_match.group(1)
                
                # Print the matched pattern
                print(f"Found pattern: {match.group()}")
                
                # If we already have some pages collected and the customer ID has changed, save the current booklet
                if current_booklet_pages and current_customer_id and current_customer_id != customer_id:
                    save_booklet(current_booklet_pages, current_customer_id)
                    current_booklet_pages = []
                
                # Update current customer ID
                current_customer_id = customer_id
        
        # Add the current page to the booklet
        current_booklet_pages.append(i)
    
    # Save the last booklet (if any)
    if current_booklet_pages and current_customer_id:
        save_booklet(current_booklet_pages, current_customer_id)
    
    doc.close()  # Close the main PDF to free resources

# Example usage
input_pdf_path = "C:/Users/dewae/Documents/jr-boekje/tibo/output/2024_09_18_15_34_53/Xmpie-Feedingplan-booklet-050924 copy4.pdf"
output_directory = "C:/Users/dewae/Documents/jr-boekje/tibo/output/2024_09_18_15_34_53/test3"
pattern = r"\d+-----\d+"  # Regex for the pattern e.g., '8392932-----238932'

split_pdf_based_on_pattern(input_pdf_path, output_directory, pattern)


Found pattern: 538832-----639991
Found pattern: 222319-----261379
Saved booklet '538832' with 1 pages.
Found pattern: 222319-----261386
Found pattern: 164543-----194786
Saved booklet '222319' with 2 pages.
Found pattern: 500728-----595799
Saved booklet '164543' with 1 pages.
Found pattern: 559443-----664007
Saved booklet '500728' with 1 pages.
Found pattern: 515352-----612820
Saved booklet '559443' with 1 pages.
Found pattern: 262305-----307190
Saved booklet '515352' with 1 pages.
Saved booklet '262305' with 1 pages.
